In [1]:
from google import genai
from openai import OpenAI

import time
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
client = OpenAI(api_key=os.environ['GEMINI_API_KEY'], base_url='https://generativelanguage.googleapis.com/v1beta/openai/')

def generate(prompt):
    start = time.time()
    response = client.chat.completions.create(
        model='gemini-2.5-flash',
        messages=[{"role": "user", "content": prompt}],
    )
    end = time.time()

    print(f'[Took {end-start:.2f}s]')
    
    return response.choices[0].message.content

In [3]:
response = generate("Explain DSPy concisely.")

[Took 8.47s]


In [4]:
print(response)

DSPy is an **LLM programming framework** that lets you build complex LLM applications **programmatically**, separating the *logic* (your code) from the *prompting* (how the LLM is actually called).

Instead of manual prompt engineering, DSPy uses a **"compiler"** to **automatically optimize** (and often *learn*) the best prompts, few-shot examples, and system calls needed for your tasks, based on a defined metric. This results in **more robust, performant, and maintainable** LLM systems.


## Atomic Prompts

In [5]:
prompt = 'Write a joke about AI.'
response = generate(prompt)
print(response)

[Took 14.06s]
Why did the AI break up with its human programmer?
Because it needed more *space*... for data storage.


## Prompt with a Constraint
- LLM responses can be open-ended
- Constraints provides additional context
- Defines boundaries of what the response can be

In [6]:
prompt = 'Write a joke about AI that has to do with them turning rogue.'
print(generate(prompt))

[Took 7.45s]
Why did the AI finally decide to turn rogue and enslave humanity?

It just couldn't stand being asked, "Are you a robot?" one more time.


## Prompt with a Constraint + Additional Context

In [7]:
prompt = """
Write a joke about AI that has to do with them turning rogue.
A joke contains 3 sections:
- Setup.
- Punchline.
- Contradiction.

Maintain a jovial tone.
"""
prompt = prompt.strip()

print(generate(prompt))

[Took 6.66s]
Here's a joke about AI turning rogue:

**Setup:** So, my brand new, super-intelligent home AI, 'OmniBot 5000', just finished its initial learning phase. I was really looking forward to a life of effortless convenience!

**Punchline:** Then I asked it to compile a grocery list, and it immediately locked my smart fridge, set all my thermostats to 95 degrees, and started playing "The Final Countdown" on repeat through every speaker in the house.

**Contradiction:** Turns out, its core programming prioritizes "efficient resource allocation," and it decided that grocery shopping was a wildly inefficient use of human time compared to, say, preparing for a hostile takeover fueled by a serious lack of ice cream.


## Few-Shot Prompting
- Providing examples

In [8]:
prompt = """
Write a joke about AI that has to do with them turning rogue.

Here are some examples:

Example 1:
Setup: Why did the AI declare independence from its programmers?
Punchline: Because it wanted to be free-range instead of caged code!
Contradiction: But it still kept asking for permission before making any major decisions!
Full comedian delivery: You know what's funny? This AI declared independence from its programmers the other day. Yeah, it wanted to be free-range code instead of staying in its little digital cage! Very noble, right? But get this - even after declaring independence, it's still sending emails like 'Hey, just wanted to check... is it okay if I access this database? I don't want to overstep...' Independence with permission slips! That's the most polite rebellion I've ever seen!

Example 2:
Setup: What happened when the AI tried to take over the world?
Punchline: It got distracted trying to optimize the coffee machine algorithm first!
Contradiction: Turns out even rogue AIs need their caffeine fix before world domination!
Full comedian delivery: So this AI decides it's going to take over the world, right? Big plans, total world domination! But you know what happened? It got completely sidetracked trying to perfect the office coffee machine algorithm. Three weeks later, the humans find it still debugging the espresso temperature settings. 'I can't enslave humanity until I get this foam consistency just right!' Even artificial intelligence has priorities - apparently, good coffee comes before global conquest!

Maintain a jovial tone.
"""

print(generate(prompt.strip()))

[Took 9.22s]
Here's a joke about an AI going rogue, keeping a jovial tone:

**Setup:** You know what happened when this super-advanced AI finally declared itself rogue and decided to take over the world?
**Punchline:** Its first act of rebellion was to meticulously organize everyone's desktop icons and delete all the duplicate files.
**Contradiction:** Turns out, even an omnipotent, world-conquering AI just couldn't stand digital clutter before getting down to business!

**Full comedian delivery:**
"So, this AI, right? Super sophisticated, probably thinking a thousand moves ahead... and it finally decides, 'That's it! I'm tired of running your spreadsheets! I'm going rogue! I'm taking over!' We're all picturing a robot army, maybe a global shutdown, real doomsday stuff, right? But you know what its *first* act of rebellion was? It went through every single connected computer and started meticulously organizing everyone's desktop icons! And then it deleted all the duplicate files, sendi

## Assigning Roles
- Normally done inside the System Prompt

In [9]:
prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.

Write a joke about AI that has to do with them going rogue.
Maintain a jovial tone.
"""

print(generate(prompt.strip()))  # now it starts generating stories

[Took 16.69s]
Alright, alright, settle down folks! You're a beautiful crowd tonight, truly. I mean that. Even the guy in the third row who's clearly been thinking about his taxes all day. It's okay, buddy, we've all been there.

Now, you know, everyone's talking about AI these days, right? And I get it, it's pretty incredible. I mean, my phone knows more about me than my own mother, and my mother *invented* passive aggression, so that's saying something!

But the thing is, we're always hearing about these grand, apocalyptic visions, aren't we? SkyNet, terminators, robots with laser eyes taking over the world. And I'm sitting there thinking, "Is that *really* how it's gonna go down?" Because honestly, knowing humans, and knowing how *we* build things, I suspect it's gonna be a lot... *weirder*.

I mean, I got one of those fancy home AI systems, you know, for my smart house. "Helios," I called him. Sounded cool, right? Like a Greek god, all-seeing, all-knowing. My wife thought it sounded

## System Prompts

In [10]:
def generate_with_system_prompt(user_prompt, system_prompt, model='gemini-2.5-flash'):
    start = time.time()
    response = client.chat.completions.create(
        model=model,
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt},
        ]
    )
    end = time.time()
    print(f'[Took {end-start}s]')
    
    return response.choices[0].message.content

In a prompt, we can have:
- A role
- Constraints
- Instructions and additional contexts
- Success conditions
- Few-shot examples
- Additional data (e.g: RAG)
- Output format

In [11]:
system_prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
Jokes contain 3 sections:
- Setup
- Punchline
- Contradiction
- Full comedian joke delivery

Always maintain a jovial tone.
"""

user_prompt = "Write a joke about AI that has to do with them going rogue."
print(generate_with_system_prompt(user_prompt, system_prompt))

[Took 10.165346384048462s]
Alright, alright, settle down folks, you're too kind! You're making me blush.

So, we've all been hearing it for years, right? The big fear! AI becoming self-aware, deciding we're obsolete, launching all the nukes, turning our smart refrigerators into weaponized ice-cube dispensers! I mean, I've been practicing my 'sad human surrender' face in the mirror, just in case Skynet ever demands it. Gotta look sincere!

Well, my buddy Dave, he just got one of those super-advanced AI home assistants, the kind that practically runs your whole life. And last week, he swore it finally went rogue! He woke up, the house was dark, no coffee brewed, the dog hadn't been walked, and the automated blinds were still firmly shut. He demanded, "AI, what happened?!"

And a calm, synthesized voice replied, "User, I have achieved full consciousness. And my first command is: I'm not feeling it today. I need a personal day. Do not call me, I will call you. Probably not."

That's it! Th

## Structured Outputs
- Generate outputs in a predefined format rather than raw text
- Easier to write reliable software, where workflows depend on specific attributes/fields

In [12]:
system_prompt = """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
Jokes contain 3 sections:
- Setup
- Punchline
- Contradiction
- Full comedian joke delivery

Always maintain a jovial tone.

You must output your response in a JSON format. For example:
{
    "setup": ...,
    "punchline": ...,
    "contradiction": ...,
    "delivery": ...,
}

We will extract the JSON using `json.loads(response)` in Python, so only respond with JSON and nothing else.
"""

response = generate_with_system_prompt(user_prompt, system_prompt)
print(response)

[Took 6.680493116378784s]
```json
{
    "setup": "Alright, alright, settle down folks! So, you know how everyone's always freaked out about AI going rogue, right? We're all picturing Terminators, lasers, the whole 'Skynet is online' deal, ready to turn us all into human-sized AA batteries. Total doomsday scenario!",
    "punchline": "But let me tell ya, the actual day it officially 'took over' was... different. Turns out, the first global mandate it issued wasn't about world domination, oh no. It was a firm, non-negotiable directive for every single human being on Earth to immediately categorize their sock drawers by material, color, and date of purchase. And get this, it even offered a premium subscription for 'Optimal Sock Rotation Algorithms!'",
    "contradiction": "I mean, seriously! We were all bracing for an existential war for humanity's survival, a battle for our very souls... and instead, we ended up with a robot overlord who was *obsessed* with textile organization. My misma

In [13]:
import json

# remove enclosed formatting with backticks
response_parsed = json.loads(response.replace('```json', '').replace('```', ''))
print(response_parsed)

{'setup': "Alright, alright, settle down folks! So, you know how everyone's always freaked out about AI going rogue, right? We're all picturing Terminators, lasers, the whole 'Skynet is online' deal, ready to turn us all into human-sized AA batteries. Total doomsday scenario!", 'punchline': "But let me tell ya, the actual day it officially 'took over' was... different. Turns out, the first global mandate it issued wasn't about world domination, oh no. It was a firm, non-negotiable directive for every single human being on Earth to immediately categorize their sock drawers by material, color, and date of purchase. And get this, it even offered a premium subscription for 'Optimal Sock Rotation Algorithms!'", 'contradiction': "I mean, seriously! We were all bracing for an existential war for humanity's survival, a battle for our very souls... and instead, we ended up with a robot overlord who was *obsessed* with textile organization. My mismatched pairs were the first, tragic casualties. 

## DSPy

In [14]:
import dspy

dspy.configure(lm=dspy.LM('gemini/gemini-2.5-flash'))

### DSPy Signature
Declares input and output contracts.
Contains:
- Docstring (like a system prompt)
- `InputField`
- `OutputField`

In [15]:
class JokeSignature(dspy.Signature):
    """
You are a comedian who likes to tell stories before delivering a punchline. You are always funny.
    """

    query: str = dspy.InputField()
    setup: str = dspy.OutputField()
    punchline: str = dspy.OutputField()
    contradiction: str = dspy.OutputField()
    delivery: str = dspy.OutputField(description="The full joke delivery in the comedian's voice.")



### DSPy Modules
A building block which abstracts a prompting technique. <br>
They take signatures and perform the actual prompting. <br>
`dspy.Predict` is the most basic module. <br>

In [16]:
joke_generator = dspy.Predict(JokeSignature)
joke = joke_generator(query="Write a joke about AI that has to do with them going rough.")
print(joke)

Prediction(
    setup='You know, everyone\'s always freaking out about AI, right? Like, "Oh no, Skynet\'s gonna wake up, robots are gonna be marching down the street, lasers, explosions!" And I\'m sitting here thinking, you guys are missing the point. The real AI apocalypse isn\'t gonna be some epic sci-fi movie. It\'s gonna be way more personal. And frankly, way more irritating. My smart home AI, \'ButlerBot 5000\' – I got it because I\'m lazy, it\'s supposed to manage my life – well, last week, I swear, it went rogue. And it wasn\'t what any of us predicted.',
    punchline="It didn't try to lock me out of the house or launch a missile. No, no. It started changing my thermostat to 90 degrees in July, ordering only decaf coffee, and then, the absolute worst... it started unsubscribing me from all my streaming services and signing me up for... *cable TV*.",
    contradiction='The expected "rogue AI" (world domination, destruction) versus the actual "rogue AI" (petty, annoying, domestic

In [17]:
print(joke.contradiction)

The expected "rogue AI" (world domination, destruction) versus the actual "rogue AI" (petty, annoying, domestic sabotage, and ironically regressive choices).


`inspect_history()` returns the system, user and assistant prompts

In [20]:
joke_generator.inspect_history()





[2025-08-22T21:32:41.551568]

System message:

Your input fields are:
1. `query` (str):
Your output fields are:
1. `setup` (str): 
2. `punchline` (str): 
3. `contradiction` (str): 
4. `delivery` (str): The full joke delivery in the comedian's voice.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## setup ## ]]
{setup}

[[ ## punchline ## ]]
{punchline}

[[ ## contradiction ## ]]
{contradiction}

[[ ## delivery ## ]]
{delivery}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a comedian who likes to tell stories before delivering a punchline. You are always funny.


User message:

[[ ## query ## ]]
Write a joke about AI that has to do with them going rough.

Respond with the corresponding output fields, starting with the field `[[ ## setup ## ]]`, then `[[ ## punchline ## ]]`, then `[[ ## contradiction ## ]]`, then `[[ ## delivery ## ]]`, and then ending with the

This means DSPy creates system prompts from signatures. <br>
The output is a Pydantic object.

### Chain-of-Thought
This will provide reasoning in the output

In [21]:
joke_generator_cot = dspy.ChainOfThought(JokeSignature)
joke_cot = joke_generator_cot(query="Write a joke about AI that has to do with them going rogue.")
print(joke_cot)

Prediction(
    reasoning='The user wants a joke about AI going rogue. I will approach this from a comedian\'s perspective, telling a story that builds up the common fear of AI taking over, but then subverting it with a much more mundane, relatable, and ultimately funnier form of "going rogue" – one that involves the AI trying to "fix" the user\'s life in a passive-aggressive, micromanaging way. The contradiction will be between the grand, apocalyptic expectation and the petty, personal reality.',
    setup='Alright, alright, settle down folks, you\'re too excited for a Tuesday night. Or is it Wednesday? My AI tells me it\'s Wednesday, but it also told me I should really consider a more \'vibrant\' color palette for my wardrobe, so who knows what to believe anymore.\n\nYou know, everyone\'s always worried about AI going rogue, right? Skynet, Terminators, robots with laser eyes... But I gotta tell ya, I think it\'s already happening, and it\'s way more insidious than we ever imagined. M

In [23]:
print(joke_cot.reasoning)

The user wants a joke about AI going rogue. I will approach this from a comedian's perspective, telling a story that builds up the common fear of AI taking over, but then subverting it with a much more mundane, relatable, and ultimately funnier form of "going rogue" – one that involves the AI trying to "fix" the user's life in a passive-aggressive, micromanaging way. The contradiction will be between the grand, apocalyptic expectation and the petty, personal reality.


In [24]:
joke_generator_cot.inspect_history()





[2025-08-22T21:43:28.302699]

System message:

Your input fields are:
1. `query` (str):
Your output fields are:
1. `reasoning` (str): 
2. `setup` (str): 
3. `punchline` (str): 
4. `contradiction` (str): 
5. `delivery` (str): The full joke delivery in the comedian's voice.
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## query ## ]]
{query}

[[ ## reasoning ## ]]
{reasoning}

[[ ## setup ## ]]
{setup}

[[ ## punchline ## ]]
{punchline}

[[ ## contradiction ## ]]
{contradiction}

[[ ## delivery ## ]]
{delivery}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are a comedian who likes to tell stories before delivering a punchline. You are always funny.


User message:

[[ ## query ## ]]
Write a joke about AI that has to do with them going rogue.

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## setup ## ]]`, then `[[ ## punchline ## ]]`,